In [4]:
# Se importan las librerias 

import yfinance as yf
import pandas as pd
import numpy as np
from pypfopt import risk_models, expected_returns, plotting, objective_functions

In [2]:
# Lista de tickers
tickers = ['AAPL', 'AMZN', 'BBVA.MC', 'BIMBOA.MX', 'GOOG', 'GRUMAB.MX', 'IBE.MC', 
           'META', 'MSFT', 'NVDA', 'PETR4.SA', 'SHOP', 'TSLA', 'WALMEX.MX']

# Descargar datos de Yahoo Finance
data = yf.download(tickers, start='2018-05-01', end='2024-05-01', interval='1d')['Adj Close']

data

[*********************100%%**********************]  14 of 14 completed


Ticker,AAPL,AMZN,BBVA.MC,BIMBOA.MX,GOOG,GRUMAB.MX,IBE.MC,META,MSFT,NVDA,PETR4.SA,SHOP,TSLA,WALMEX.MX
Date,,,,,,,,,,,,,,
2018-05-01,40.033375,79.112999,NaN,NaN,51.865501,NaN,NaN,173.675735,88.768616,56.219429,NaN,12.768000,19.994667,NaN
2018-05-02,41.801842,78.484001,4.872661,38.493103,51.219002,202.112625,5.059384,175.883392,87.376350,56.013985,7.677155,12.481000,20.076668,48.118729
2018-05-03,41.877613,78.603996,4.801354,37.606407,51.186001,200.868774,5.051632,173.835556,87.899620,57.667358,7.626202,13.517000,18.963333,47.685230
2018-05-04,43.520615,79.047501,4.856096,37.652119,52.410500,195.311142,5.093496,176.422821,88.918114,59.169743,7.565056,13.613000,19.606001,47.657555
2018-05-07,43.835476,80.007004,4.889228,37.167645,52.739498,193.246872,5.093496,177.781372,89.908600,61.550793,7.694139,13.856000,20.184668,47.168713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-24,168.791061,176.589996,10.550000,68.363243,161.100006,340.709991,11.530000,493.500000,408.323486,796.770020,38.437523,72.260002,162.130005,65.919998
2024-04-25,169.659882,173.669998,10.505000,69.449005,157.949997,338.100006,11.460000,441.380005,398.321533,826.320007,39.360474,70.550003,170.179993,67.669998
2024-04-26,169.070679,179.619995,10.985000,67.800629,173.690002,345.480011,11.610000,443.290009,405.588440,877.349976,39.908001,71.330002,168.289993,66.650002


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1553 entries, 2018-05-01 to 2024-04-30
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AAPL       1510 non-null   float64
 1   AMZN       1510 non-null   float64
 2   BBVA.MC    1536 non-null   float64
 3   BIMBOA.MX  1511 non-null   float64
 4   GOOG       1510 non-null   float64
 5   GRUMAB.MX  1511 non-null   float64
 6   IBE.MC     1536 non-null   float64
 7   META       1510 non-null   float64
 8   MSFT       1510 non-null   float64
 9   NVDA       1510 non-null   float64
 10  PETR4.SA   1488 non-null   float64
 11  SHOP       1510 non-null   float64
 12  TSLA       1510 non-null   float64
 13  WALMEX.MX  1511 non-null   float64
dtypes: float64(14)
memory usage: 182.0 KB


In [5]:
# Calcular los rendimientos diarios
returns = data.pct_change().dropna()
returns

Ticker,AAPL,AMZN,BBVA.MC,BIMBOA.MX,GOOG,GRUMAB.MX,IBE.MC,META,MSFT,NVDA,PETR4.SA,SHOP,TSLA,WALMEX.MX
Date,,,,,,,,,,,,,,
2018-05-03,0.001813,0.001529,-0.014634,-0.023035,-0.000644,-0.006154,-0.001532,-0.011643,0.005989,0.029517,-0.006637,0.083006,-0.055454,-0.009009
2018-05-04,0.039233,0.005642,0.011401,0.001216,0.023923,-0.027668,0.008287,0.014883,0.011587,0.026053,-0.008018,0.007102,0.033890,-0.000580
2018-05-07,0.007235,0.012138,0.006823,-0.012867,0.006277,-0.010569,0.000000,0.007701,0.011139,0.040241,0.017063,0.017851,0.029515,-0.010257
2018-05-08,0.004807,-0.004843,-0.004714,0.004427,-0.000834,-0.004428,-0.003044,0.005338,-0.004261,0.006916,0.011479,0.034281,-0.002642,-0.013492
2018-05-09,0.007041,0.009803,0.007401,-0.030607,0.027374,-0.011234,-0.001832,0.020903,0.011794,0.021486,0.081624,0.019468,0.016161,-0.018434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-24,0.012702,-0.016431,-0.010319,0.002025,0.007379,-0.005662,-0.000867,-0.005241,0.003656,-0.033316,-0.004587,-0.023645,0.120611,0.013686
2024-04-25,0.005147,-0.016535,-0.004265,0.015882,-0.019553,-0.007660,-0.006071,-0.105613,-0.024495,0.037087,0.024012,-0.023665,0.049651,0.026547
2024-04-26,-0.003473,0.034260,0.045692,-0.023735,0.099652,0.021828,0.013089,0.004327,0.018244,0.061756,0.013911,0.011056,-0.011106,-0.015073


In [7]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2018-05-03 to 2024-04-30
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AAPL       1551 non-null   float64
 1   AMZN       1551 non-null   float64
 2   BBVA.MC    1551 non-null   float64
 3   BIMBOA.MX  1551 non-null   float64
 4   GOOG       1551 non-null   float64
 5   GRUMAB.MX  1551 non-null   float64
 6   IBE.MC     1551 non-null   float64
 7   META       1551 non-null   float64
 8   MSFT       1551 non-null   float64
 9   NVDA       1551 non-null   float64
 10  PETR4.SA   1551 non-null   float64
 11  SHOP       1551 non-null   float64
 12  TSLA       1551 non-null   float64
 13  WALMEX.MX  1551 non-null   float64
dtypes: float64(14)
memory usage: 181.8 KB


In [10]:
market_prices = yf.download("SPY", start="2018-05-01", end = "2024-05-01")['Adj Close']
market_prices

[*********************100%%**********************]  1 of 1 completed


Date
2018-05-01    239.859161
2018-05-02    238.247925
2018-05-03    237.722916
2018-05-04    240.800629
2018-05-07    241.615311
                 ...    
2024-04-24    505.410004
2024-04-25    503.489990
2024-04-26    508.260010
2024-04-29    510.059998
2024-04-30    501.980011
Name: Adj Close, Length: 1510, dtype: float64

In [13]:
market_return = expected_returns.mean_historical_return(market_prices)
market_return

c:\Users\Alvaro\AppData\Local\Programs\Python\Python311\Lib\site-packages\pypfopt\expected_returns.py:137: RuntimeWarning: prices are not in a dataframe
  warnings.warn("prices are not in a dataframe", RuntimeWarning)


Adj Close    0.131257
dtype: float64

In [9]:
# Descargar el beta de Yahoo Finance
betas = {}
for ticker in tickers:
    ticker_data = yf.Ticker(ticker)
    betas[ticker] = ticker_data.info['beta']

betas

{'AAPL': 1.264,
 'AMZN': 1.155,
 'BBVA.MC': 1.49,
 'BIMBOA.MX': 0.211,
 'GOOG': 1.019,
 'GRUMAB.MX': 0.366,
 'IBE.MC': 0.568,
 'META': 1.208,
 'MSFT': 0.893,
 'NVDA': 1.749,
 'PETR4.SA': 1.006,
 'SHOP': 2.199,
 'TSLA': 2.408,
 'WALMEX.MX': 0.12}

In [14]:
risk_free_rate = 0.04357

In [15]:
capm = {}
for ticker, beta in betas.items():
    expected_return = risk_free_rate + beta * (market_return - risk_free_rate)
    capm[ticker] = expected_return

capm

{'AAPL': Adj Close    0.154406
 dtype: float64,
 'AMZN': Adj Close    0.144849
 dtype: float64,
 'BBVA.MC': Adj Close    0.174224
 dtype: float64,
 'BIMBOA.MX': Adj Close    0.062072
 dtype: float64,
 'GOOG': Adj Close    0.132923
 dtype: float64,
 'GRUMAB.MX': Adj Close    0.075663
 dtype: float64,
 'IBE.MC': Adj Close    0.093376
 dtype: float64,
 'META': Adj Close    0.149496
 dtype: float64,
 'MSFT': Adj Close    0.121875
 dtype: float64,
 'NVDA': Adj Close    0.196935
 dtype: float64,
 'PETR4.SA': Adj Close    0.131783
 dtype: float64,
 'SHOP': Adj Close    0.236394
 dtype: float64,
 'TSLA': Adj Close    0.25472
 dtype: float64,
 'WALMEX.MX': Adj Close    0.054092
 dtype: float64}

In [17]:
# Pesos del portafolio dados
weights = {
    'AAPL': 0.10459,
    'AMZN': 0.06597,
    'BBVA.MC': 0.02476,
    'BIMBOA.MX': 0.01537,
    'GOOG': 0.11496,
    'GRUMAB.MX': 0.03467,
    'IBE.MC': 0.01145,
    'META': 0.07591,
    'MSFT': 0.17819,
    'NVDA': 0.19194,
    'PETR4.SA': 0.04367,
    'SHOP': 0.04889,
    'TSLA': 0.06273,
    'WALMEX.MX': 0.02692
}

# Calcular el CAPM del portafolio
portfolio_capm = sum(weights[ticker] * capm[ticker] for ticker in tickers)

portfolio_capm


Adj Close    0.155558
dtype: float64

In [9]:
# Define los pesos de las inversiones
peso_renta_variable = 0.45
peso_renta_fija = 0.55

# Define los rendimientos esperados
rendimiento_renta_variable = 0.196  # 15.456%
rendimiento_renta_fija = 0.04357     # 4.345%

# Calcula el rendimiento esperado del portafolio
rendimiento_esperado_portafolio = (peso_renta_variable * rendimiento_renta_variable) + (peso_renta_fija * rendimiento_renta_fija)

rendimiento_esperado_portafolio

0.1121635

In [10]:
import math


# Define la desviación estándar (riesgo) de cada componente
desviacion_estandar_renta_variable = 0.259  # 25.9%
desviacion_estandar_renta_fija = 0.0        # Suponemos que es despreciable

# Define la correlación entre los componentes
correlacion = 0.0  # Suponemos que es cero

# Calcula la varianza del portafolio
varianza_portafolio = (peso_renta_variable**2 * desviacion_estandar_renta_variable**2) + \
                      (peso_renta_fija**2 * desviacion_estandar_renta_fija**2) + \
                      (2 * peso_renta_variable * peso_renta_fija * desviacion_estandar_renta_variable * desviacion_estandar_renta_fija * correlacion)

# Calcula la desviación estándar del portafolio
desviacion_estandar_portafolio = math.sqrt(varianza_portafolio)

desviacion_estandar_portafolio


0.11655